<a href="https://colab.research.google.com/github/Santosdevbjj/analiseRiscosAtrasoObras/blob/main/Notebooks/02_modelagem_preditiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ============================================================
# 1. Clonar o repositório e acessar a pasta
# ============================================================
# !git clone https://github.com/Santosdevbjj/analiseRiscosAtrasoObras.git
# %cd analiseRiscosAtrasoObras

# ============================================================
# 2. Imports
# ============================================================
import pandas as pd

# ============================================================
# 3. Carregamento dos CSVs
# ============================================================
atividades   = pd.read_csv("data/raw/atividades.csv")
fornecedores = pd.read_csv("data/raw/fornecedores.csv")
obras        = pd.read_csv("data/raw/obras.csv")
suprimentos  = pd.read_csv("data/raw/suprimentos.csv")

# Visualizar as primeiras linhas de cada dataset
print("Atividades:")
print(atividades.head(), "\n")

print("Fornecedores:")
print(fornecedores.head(), "\n")

print("Obras:")
print(obras.head(), "\n")

print("Suprimentos:")
print(suprimentos.head(), "\n")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/atividades.csv'

In [ ]:

# ============================================================
# 1. Imports e Setup
# ============================================================
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [ ]:

# ============================================================
# 2. Carregamento dos Dados
# ============================================================
atividades   = pd.read_csv("data/raw/atividades.csv")
fornecedores = pd.read_csv("data/raw/fornecedores.csv")
obras        = pd.read_csv("data/raw/obras.csv")
suprimentos  = pd.read_csv("data/raw/suprimentos.csv")

print("Datasets carregados:")
print("atividades:", atividades.shape)
print("fornecedores:", fornecedores.shape)
print("obras:", obras.shape)
print("suprimentos:", suprimentos.shape)

In [ ]:

# ============================================================
# 3. Integração dos Dados
# ============================================================
# 1. atividades + obras
df_mestre = atividades.merge(obras, on="id_obra", how="left")

# 2. adiciona suprimentos (traz id_fornecedor)
df_mestre = df_mestre.merge(suprimentos, on=["id_obra", "id_atividade"], how="left")

# 3. adiciona fornecedores
df_mestre = df_mestre.merge(fornecedores, on="id_fornecedor", how="left")

print("df_mestre consolidado:")
print(df_mestre.head())
print("Shape:", df_mestre.shape)

In [ ]:

# ============================================================
# 4. Limpeza básica
# ============================================================
df_mestre = df_mestre.dropna(subset=["dias_atraso"]).copy()
df_mestre["rating_confiabilidade"] = df_mestre["rating_confiabilidade"].fillna(df_mestre["rating_confiabilidade"].median())
df_mestre["orcamento_estimado"] = df_mestre["orcamento_estimado"].clip(lower=0).fillna(df_mestre["orcamento_estimado"].median())

for col in ["material", "cidade", "etapa"]:
    df_mestre[col] = df_mestre[col].fillna("desconhecido")

In [ ]:

# ============================================================
# 5. Feature Engineering
# ============================================================
df_mestre = df_mestre.assign(
    taxa_insucesso_fornecedor = df_mestre.groupby("id_fornecedor")["dias_atraso"].transform(lambda x: (x > 0).mean()),
    complexidade_obra = np.log1p(df_mestre["orcamento_estimado"]),
    risco_etapa = df_mestre.groupby("etapa")["dias_atraso"].transform("mean")
)

print("Preview de features derivadas:")
print(df_mestre[[
    "id_obra","id_fornecedor","etapa","dias_atraso",
    "taxa_insucesso_fornecedor","complexidade_obra","risco_etapa"
]].head())

In [ ]:

# ============================================================
# 6. Preparação Final das Features
# ============================================================
df_model = pd.get_dummies(
    df_mestre[[
        "orcamento_estimado",
        "rating_confiabilidade",
        "material",
        "cidade",
        "etapa",
        "taxa_insucesso_fornecedor",
        "complexidade_obra",
        "risco_etapa",
        "dias_atraso"
    ]],
    columns=["material", "cidade", "etapa"]
)

X = df_model.drop("dias_atraso", axis=1)
y = df_model["dias_atraso"]

print("Shapes:")
print("X:", X.shape, "| y:", y.shape)

In [ ]:

# ============================================================
# 7. Divisão Treino/Teste e Treinamento
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

print("Modelo treinado.")

In [ ]:

# ============================================================
# 8. Avaliação do Modelo
# ============================================================
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=== Avaliação do Modelo ===")
print(f"Erro Médio Absoluto (MAE): {mae:.2f} dias")
print(f"R² Score: {r2:.2f}")

In [ ]:

# ============================================================
# 9. Interpretação Visual das Features
# ============================================================
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
feature_importance_df = pd.DataFrame({
    "Feature": X.columns[indices],
    "Importância": importances[indices]
})

plt.figure(figsize=(12,6))
sns.barplot(x="Importância", y="Feature", data=feature_importance_df, palette="viridis")
plt.title("Importância das Features no Modelo RandomForest", fontsize=14)
plt.xlabel("Importância relativa")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

print("Top 10 Features mais importantes:")
print(feature_importance_df.head(10))

In [ ]:

# ============================================================
# 10. Impacto de Negócio
# ============================================================
custo_por_dia = 50000
impacto_financeiro = mae * custo_por_dia

print("=== Impacto de Negócio ===")
print(f"Cada dia de atraso custa aproximadamente R$ {custo_por_dia:,.0f}.")
print(f"Com um erro médio de {mae:.2f} dias, o impacto financeiro potencial é de ~R$ {impacto_financeiro:,.0f}.")

In [ ]:

# ============================================================
# 11. Salvar Modelo
# ============================================================
import os
os.makedirs("models", exist_ok=True)
joblib.dump(model, "models/modelo_random_forest.pkl")
print("Modelo salvo em models/modelo_random_forest.pkl")

# 📊 Previsão de Atrasos – Versão Executiva

## 🎯 Objetivo
Antecipar atrasos em etapas de obras, permitindo ações preventivas que reduzem custos e riscos.

## 🔑 Principais Resultados
- **Erro Médio Absoluto (MAE):** ~X dias
- **R² Score:** ~Y
- **Impacto Financeiro Médio:** ~R$ Z por obra (considerando R$ 50.000/dia)

## 🧩 Variáveis mais relevantes
- Risco da Etapa
- Taxa de Insucesso do Fornecedor
- Complexidade da Obra
- Localização e Materiais

## 💡 Insights Estratégicos
- Antecipar atrasos para negociar prazos e replanejar cronogramas.
- Reduz multas e custos indiretos.
- Melhora confiabilidade da entrega e satisfação dos clientes.

## 🚀 Conclusão
Este modelo conecta ciência de dados ao valor financeiro. Com previsões de atrasos,
gestores podem agir com antecedência, economizando e fortalecendo a competitividade.

In [ ]:

# ============================================================
# 12. Simulador de Risco – Exemplo de uso do modelo
# ============================================================

# Exemplo de como usar o modelo para uma nova obra
nova_obra = {
    'orcamento_estimado': 12000000,
    'rating_confiabilidade': 2.5,
    'taxa_insucesso_fornecedor': 0.8,  # fornecedor perigoso
    'complexidade_obra': np.log1p(12000000),
    'risco_etapa': 10.0,
    'material': 'concreto',
    'cidade': 'Belo Horizonte',
    'etapa': 'Fundação'
}

# Transformar em DataFrame para prever
df_nova = pd.DataFrame([nova_obra])

# Aplicar one-hot encoding igual ao treinamento
df_nova_encoded = pd.get_dummies(df_nova, columns=["material","cidade","etapa"])

# Garantir que tenha as mesmas colunas de X (adiciona colunas faltantes com 0)
for col in X.columns:
    if col not in df_nova_encoded.columns:
        df_nova_encoded[col] = 0

# Reordenar colunas
df_nova_encoded = df_nova_encoded[X.columns]

# Fazer previsão
pred_atraso = model.predict(df_nova_encoded)[0]

print("=== Simulador de Risco ===")
print(f"Previsão de atraso para a nova obra: {pred_atraso:.2f} dias")

In [ ]:

import os

# 1. Garantir que a pasta de destino existe
path_figures = '../reports/figures'
if not os.path.exists(path_figures):
    os.makedirs(path_figures)

# 2. Salvar o Gráfico de Importância das Features
plt.figure(figsize=(12,6))
sns.barplot(x="Importância", y="Feature", data=feature_importance_df, palette="viridis")
plt.title("Importância das Features no Modelo RandomForest", fontsize=14)
plt.xlabel("Importância relativa")
plt.ylabel("Features")
plt.tight_layout()

# Salvando a imagem
plt.savefig(f'{path_figures}/feature_importance.png', dpi=300)
print(f"✅ Gráfico salvo em: {path_figures}/feature_importance.png")

# 3. Gerar Texto para o README (Métricas)
print("\n--- COPIE E COLE NO SEU README.MD ---")
markdown_metrics = f"""
| Métrica | Valor |
| :--- | :--- |
| **Erro Médio Absoluto (MAE)** | {mae:.2f} dias |
| **R² Score** | {r2:.2f} |
| **Impacto Financeiro (R$)** | R$ {impacto_financeiro:,.2f} |
"""
print(markdown_metrics)

In [ ]:

import os

# 1. Garantir que a pasta de destino existe
path_figures = '../reports/figures'
if not os.path.exists(path_figures):
    os.makedirs(path_figures)

# 2. Salvar o Gráfico de Importância das Features
plt.figure(figsize=(12,6))
sns.barplot(x="Importância", y="Feature", data=feature_importance_df, palette="viridis")
plt.title("Importância das Features no Modelo RandomForest", fontsize=14)
plt.xlabel("Importância relativa")
plt.ylabel("Features")
plt.tight_layout()

# Salvando a imagem
plt.savefig(f'{path_figures}/feature_importance.png', dpi=300)
print(f"✅ Gráfico salvo em: {path_figures}/feature_importance.png")

# 3. Gerar Texto para o README (Métricas)
print("\n--- COPIE E COLE NO SEU README.MD ---")
markdown_metrics = f"""
| Métrica | Valor |
| :--- | :--- |
| **Erro Médio Absoluto (MAE)** | {mae:.2f} dias |
| **R² Score** | {r2:.2f} |
| **Impacto Financeiro (R$)** | R$ {impacto_financeiro:,.2f} |
"""
print(markdown_metrics)